In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: to suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

In [2]:
from pandas import json_normalize

In [41]:
# fast import of the json file BUT nested structure in images is not represented correctly

df_user = pd.read_json('.\data\All_Beauty.jsonl', orient= 'records', nrows= 10000, lines= True)
df_user.head(10)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,True
1,4,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,True
2,5,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,2020-05-16 21:41:06.052,2,True
3,1,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2022-01-28 18:13:50.220,0,True
4,5,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2020-12-30 10:02:43.534,0,True
5,4,Pretty Color,The polish was quiet thick and did not apply s...,[{'small_image_url': 'https://images-na.ssl-im...,B00R8DXL44,B00R8DXL44,AGMJ3EMDVL6OWBJF7CA5RGJLXN5A,2020-08-27 22:30:08.138,0,True
6,5,Handy,Great for many tasks. I purchased these for m...,[],B099DRHW5V,B099DRHW5V,AHREXOGQPZDA6354MHH4ETSF3MCQ,2021-09-17 13:31:59.443,0,True
7,3,Meh,These were lightweight and soft but much too s...,[{'small_image_url': 'https://m.media-amazon.c...,B088SZDGXG,B08BBQ29N5,AEYORY2AVPMCPDV57CE337YU5LXA,2021-10-15 05:20:59.292,0,True
8,5,Great for at home use and so easy to use!,This is perfect for my between salon visits. I...,[],B08P2DZB4X,B08P2DZB4X,AFSKPY37N3C43SOI5IEXEK5JSIYA,2021-07-27 13:04:04.559,0,False
9,5,Nice shampoo for the money,I get Keratin treatments at the salon at least...,[],B086QY6T7N,B086QY6T7N,AFSKPY37N3C43SOI5IEXEK5JSIYA,2021-07-18 13:21:51.145,0,False


Dropping 'title', 'text', 'images' and 'asin' columns

In [42]:
df_user.drop(['title', 'text', 'images', 'asin'], axis= 1, inplace= True)
df_user.head(3)

,rating,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,True
1,4,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,True
2,5,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,2020-05-16 21:41:06.052,2,True


In [43]:
df_user.info()
print(f"\nDataFrame(rows, columns): {df_user.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   rating             10000 non-null  int64         
 1   parent_asin        10000 non-null  object        
 2   user_id            10000 non-null  object        
 3   timestamp          10000 non-null  datetime64[ns]
 4   helpful_vote       10000 non-null  int64         
 5   verified_purchase  10000 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(2)
memory usage: 400.5+ KB

DataFrame(rows, columns): (10000, 6)


In [44]:
df_user[['parent_asin', 'user_id']].nunique()

parent_asin    7067
user_id        6585
dtype: int64

In [51]:
df_user['helpful_vote'].sort_values(ascending= False)

24      430
9187    133
4382    120
5555     92
7394     63
       ... 
22        0
21        0
20        0
19        0
17        0
Name: helpful_vote, Length: 10000, dtype: int64

In [49]:
df_user.describe()

,rating,timestamp,helpful_vote
count,10000.000000,10000,10000.000000
mean,4.118300,2019-04-27 07:03:29.752212736,0.949000
min,1.000000,2003-03-02 00:30:24,0.000000
25%,4.000000,2017-09-06 08:57:12.437500160,0.000000
50%,5.000000,2019-12-18 09:31:06.434000128,0.000000
75%,5.000000,2021-02-23 16:56:46.760250112,1.000000
max,5.000000,2023-03-18 09:49:50.883000,430.000000
std,1.295405,NaN,5.722908


# Not working ways

## using json.load and json_normalize

- https://sparkbyexamples.com/pandas/pandas-read-json-with-examples/
- https://sparkbyexamples.com/pandas/pandas-convert-json-to-dataframe/
- https://www.kdnuggets.com/how-to-convert-json-data-into-a-dataframe-with-pandas
- https://www.freecodecamp.org/news/python-read-json-file-how-to-load-json-from-a-file-and-parse-dumps/


In [45]:
import json

In [46]:
# Read the JSON file
with open('./data/All_Beauty.jsonl', 'r') as f: 
    json_data = json.load(f)


JSONDecodeError: Extra data: line 2 column 1 (char 564)

In [ ]:
# Create a DataFrame using json_normalize()

df_user_flat = json_normalize(
    data= df_user,
    record_path= 'images',
    errors= 'raise'
)

df_user_flat.head(10)


TypeError: string indices must be integers, not 'str'

In [ ]:
# Flatten nested json file of 'user' data
json_file_user = '.\data\All_Beauty.jsonl'
nested_data = []
with open(json_file_user, 'r') as file:
    for line in file:
        nested_data.append(json.load(line))

df_user = json_normalize(nested_data)
df_user.to_csv(r'.\data\normalized_user.csv', index=False)
print("Nested JSON file has been flattened and saved as {df_user}.")


AttributeError: 'str' object has no attribute 'read'